In [ ]:
#Preprocessing the model

In [5]:
import os
import json
import music21 as m21
import numpy as np
#import keras
import tensorflow.keras as keras

KERN_DATASET_PATH = "yaman1"
SAVE_DIR = "dataset"
SINGLE_FILE_DATASET = "file_dataset"
MAPPING_PATH = "mapping.json"
SEQUENCE_LENGTH = 64

# durations are expressed in quarter length
ACCEPTABLE_DURATIONS = [
    0.25, # 16th note
    0.5, # 8th note
    0.75,
    1.0, # quarter note
    1.5,
    2, # half note
    3,
    4 # whole note
    
]


def load_songs_in_kern(dataset_path):
    """Loads all kern pieces in dataset using music21.
    :param dataset_path (str): Path to dataset
    :return songs (list of m21 streams): List containing all pieces
    """
    songs = []

    # go through all the files in dataset and load them with music21
    for path, subdirs, files in os.walk(dataset_path):
        for file in files:

            # consider only kern files
            if file[-4:] == "midi":
                song = m21.converter.parse(os.path.join(path, file))
                songs.append(song)
                #print(songs)
    return songs

'''
def has_acceptable_durations(song, acceptable_durations):
    """Boolean routine that returns True if piece has all acceptable duration, False otherwise.
    :param song (m21 stream):
    :param acceptable_durations (list): List of acceptable duration in quarter length
    :return (bool):
    """
    for note in song.flat.notesAndRests:
        if note.duration.quarterLength not in acceptable_durations:
            return False
        #print(has_acceptable_durations(song,ACCEPTABLE_DURATIONS))
    return True
'''


def transpose(song):
    """Transposes song to C maj/A min
    :param piece (m21 stream): Piece to transpose
    :return transposed_song (m21 stream):
    """

    # get key from the song
    parts = song.getElementsByClass(m21.stream.Part)
    measures_part0 = parts[0].getElementsByClass(m21.stream.Measure)
    key = measures_part0[0][4]

    # estimate key using music21
    if not isinstance(key, m21.key.Key):
        key = song.analyze("key")

    # get interval for transposition. E.g., Bmaj -> Cmaj
    if key.mode == "major":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C"))
    elif key.mode == "minor":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A"))

    # transpose song by calculated interval
    tranposed_song = song.transpose(interval)
    return tranposed_song


def encode_song(song):
    """Converts a score into a time-series-like music representation. Each item in the encoded list represents 'min_duration'
    quarter lengths. The symbols used at each step are: integers for MIDI notes, 'r' for representing a rest, and '_'
    for representing notes/rests that are carried over into a new time step. Here's a sample encoding:
        ["r", "_", "60", "_", "_", "_", "72" "_"]
    :param song (m21 stream): Piece to encode
    :param time_step (float): Duration of each time step in quarter length
    :return:
    """

    encoded_song = []

    for event in song.flat.notesAndRests:

        # handle notes
        if isinstance(event, m21.note.Note):
            symbol = event.pitch.midi # 60
        # handle rests
        elif isinstance(event, m21.note.Rest):
            symbol = "r"

        # convert the note/rest into time series notation
        steps = int(event.duration.quarterLength )
        for step in range(steps):

            # if it's the first time we see a note/rest, let's encode it. Otherwise, it means we're carrying the same
            # symbol in a new time step
            if step == 0 :

                encoded_song.append(symbol)
                
            else:
                encoded_song.append("_")

    # cast encoded song to str
    encoded_song = " ".join(map(str, encoded_song))

    return encoded_song


def preprocess(dataset_path):

    # load folk songs
    print("Loading songs...")
    songs = load_songs_in_kern(dataset_path)
    print(f"Loaded {len(songs)} songs.")

    for i, song in enumerate(songs):

        # filter out songs that have non-acceptable durations
        #if not has_acceptable_durations(song, ACCEPTABLE_DURATIONS):
            #continue

        # transpose songs to Cmaj/Amin
        song = transpose(song)

        # encode songs with music time series representation
        encoded_song = encode_song(song)

        # save songs to text file
        save_path = os.path.join(SAVE_DIR, str(i))
        with open(save_path, "w") as fp:
            fp.write(encoded_song)

        if i % 10 == 0:
            print(f"Song {i} out of {len(songs)} processed")


def load(file_path):
    with open(file_path, "r") as fp:
        song = fp.read()
    return song


def create_single_file_dataset(dataset_path, file_dataset_path, sequence_length):
    """Generates a file collating all the encoded songs and adding new piece delimiters.
    :param dataset_path (str): Path to folder containing the encoded songs
    :param file_dataset_path (str): Path to file for saving songs in single file
    :param sequence_length (int): # of time steps to be considered for training
    :return songs (str): String containing all songs in dataset + delimiters
    """

    new_song_delimiter = "/ " * sequence_length
    songs = ""

    # load encoded songs and add delimiters
    for path, _, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(path, file)
            song = load(file_path)
            songs = songs + song + " " + new_song_delimiter

    # remove empty space from last character of string
    songs = songs[:-1]

    # save string that contains all the dataset
    with open(file_dataset_path, "w") as fp:
        fp.write(songs)

    return songs


def create_mapping(songs, mapping_path):
    """Creates a json file that maps the symbols in the song dataset onto integers
    :param songs (str): String with all songs
    :param mapping_path (str): Path where to save mapping
    :return:
    """
    mappings = {}

    # identify the vocabulary
    songs = songs.split()
    vocabulary = list(set(songs))

    # create mappings
    for i, symbol in enumerate(vocabulary):
        mappings[symbol] = i

    # save voabulary to a json file
    with open(mapping_path, "w") as fp:
        json.dump(mappings, fp, indent=4)


def convert_songs_to_int(songs):
    int_songs = []

    # load mappings
    with open(MAPPING_PATH, "r") as fp:
        mappings = json.load(fp)

    # transform songs string to list
    songs = songs.split()

    # map songs to int
    for symbol in songs:
        int_songs.append(mappings[symbol])

    return int_songs


def generate_training_sequences(sequence_length):
    """Create input and output data samples for training. Each sample is a sequence.
    :param sequence_length (int): Length of each sequence. With a quantisation at 16th notes, 64 notes equates to 4 bars
    :return inputs (ndarray): Training inputs
    :return targets (ndarray): Training targets
    """

    # load songs and map them to int
    songs = load(SINGLE_FILE_DATASET)
    int_songs = convert_songs_to_int(songs)

    inputs = []
    targets = []

    # generate the training sequences
    num_sequences = len(int_songs) - sequence_length
    for i in range(num_sequences):
        inputs.append(int_songs[i:i+sequence_length])
        targets.append(int_songs[i+sequence_length])

    # one-hot encode the sequences
    vocabulary_size = len(set(int_songs))
    # inputs size: (# of sequences, sequence length, vocabulary size)
    inputs = keras.utils.to_categorical(inputs, num_classes=vocabulary_size)
    targets = np.array(targets)

    print(f"There are {len(inputs)} sequences.")

    return inputs, targets


def main():
    preprocess(KERN_DATASET_PATH)
    songs = create_single_file_dataset(SAVE_DIR, SINGLE_FILE_DATASET, SEQUENCE_LENGTH)
    create_mapping(songs, MAPPING_PATH)
    #inputs, targets = generate_training_sequences(SEQUENCE_LENGTH)



In [6]:
main()

Loading songs...
Loaded 60 songs.
Song 0 out of 60 processed
Song 10 out of 60 processed
Song 20 out of 60 processed
Song 30 out of 60 processed
Song 40 out of 60 processed
Song 50 out of 60 processed


In [7]:
import tensorflow.keras as keras
#from preprocess import generate_training_sequences, SEQUENCE_LENGTH

OUTPUT_UNITS = 53
NUM_UNITS = [256]
LOSS = "sparse_categorical_crossentropy"
LEARNING_RATE = 0.001
EPOCHS = 30
BATCH_SIZE = 64
SAVE_MODEL_PATH = "model.h5"


def build_model(output_units, num_units, loss, learning_rate):
    """Builds and compiles model
    :param output_units (int): Num output units
    :param num_units (list of int): Num of units in hidden layers
    :param loss (str): Type of loss function to use
    :param learning_rate (float): Learning rate to apply
    :return model (tf model): Where the magic happens :D
    """

    # create the model architecture
    
    input = keras.layers.Input(shape=(None, output_units))
    x1 = keras.layers.LSTM(num_units[0],return_sequences=True)(input)
    x2 = keras.layers.LSTM(num_units[0],return_sequences=True)(x1)
    x3 = keras.layers.LSTM(num_units[0],return_sequences=True)(x2)
    x4 = keras.layers.LSTM(num_units[0])(x3)
    x5 = keras.layers.Dropout(0.2)(x4)

    output = keras.layers.Dense(output_units, activation="softmax")(x5)
    model = keras.Model(input, output)

    # compile model
    model.compile(loss=loss,
                  optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=["accuracy"])

    model.summary()

    return model


def train(output_units=OUTPUT_UNITS, num_units=NUM_UNITS, loss=LOSS, learning_rate=LEARNING_RATE):
    """Train and save TF model.
    :param output_units (int): Num output units
    :param num_units (list of int): Num of units in hidden layers
    :param loss (str): Type of loss function to use
    :param learning_rate (float): Learning rate to apply
    """

    # generate the training sequences
    inputs, targets = generate_training_sequences(SEQUENCE_LENGTH)

    # build the network
    model = build_model(output_units, num_units, loss, learning_rate)

    # train the model
    model.fit(inputs, targets, epochs=EPOCHS, batch_size=BATCH_SIZE)

    # save the model
    model.save(SAVE_MODEL_PATH)



In [8]:
train()

There are 10203 sequences.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 53)]        0         
                                                                 
 lstm (LSTM)                 (None, None, 256)         317440    
                                                                 
 lstm_1 (LSTM)               (None, None, 256)         525312    
                                                                 
 lstm_2 (LSTM)               (None, None, 256)         525312    
                                                                 
 lstm_3 (LSTM)               (None, 256)               525312    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None

In [7]:
import os
import json
import music21 as m21
import numpy as np
#import keras
import tensorflow.keras as keras
from music21 import *
import IPython
from keras.models import load_model
KERN_DATASET_PATH = "yaman1"
SAVE_DIR = "dataset"
SINGLE_FILE_DATASET = "file_dataset"
MAPPING_PATH = "mapping.json"
SEQUENCE_LENGTH = 64
model = load_model('model.h5')


In [8]:
from music21 import *
def sitarmusic(mel):
    s = converter.parse('mel.mid')

    for el in s.recurse():
         if 'Instrument' in el.classes: # or 'Piano'
             el.activeSite.replace(el, instrument.Sitar())

    s.write('midi', 'mel.mid')

In [9]:
import json
import numpy as np
import tensorflow.keras as keras
import music21 as m21
#from preprocess import SEQUENCE_LENGTH, MAPPING_PATH

class MelodyGenerator:
    """A class that wraps the LSTM model and offers utilities to generate melodies."""

    def __init__(self, model_path="model.h5"):
        """Constructor that initialises TensorFlow model"""

        self.model_path = model_path
        self.model = keras.models.load_model(model_path)

        with open(MAPPING_PATH, "r") as fp:
            self._mappings = json.load(fp)

        self._start_symbols = ["/"] * SEQUENCE_LENGTH


    def generate_melody(self, seed, num_steps, max_sequence_length, temperature):
        """Generates a melody using the DL model and returns a midi file.
        :param seed (str): Melody seed with the notation used to encode the dataset
        :param num_steps (int): Number of steps to be generated
        :param max_sequence_len (int): Max number of steps in seed to be considered for generation
        :param temperature (float): Float in interval [0, 1]. Numbers closer to 0 make the model more deterministic.
            A number closer to 1 makes the generation more unpredictable.
        :return melody (list of str): List with symbols representing a melody
        """

        # create seed with start symbols
        seed = seed.split()
        melody = seed
        seed = self._start_symbols + seed

        # map seed to int
        seed = [self._mappings[symbol] for symbol in seed]

        for _ in range(num_steps):

            # limit the seed to max_sequence_length
            seed = seed[-max_sequence_length:]

            # one-hot encode the seed
            onehot_seed = keras.utils.to_categorical(seed, num_classes=len(self._mappings))
            # (1, max_sequence_length, num of symbols in the vocabulary)
            onehot_seed = onehot_seed[np.newaxis, ...]

            # make a prediction
            probabilities = self.model.predict(onehot_seed)[0]
            # [0.1, 0.2, 0.1, 0.6] -> 1
            output_int = self._sample_with_temperature(probabilities, temperature)

            # update seed
            seed.append(output_int)

            # map int to our encoding
            output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]

            # check whether we're at the end of a melody
            if output_symbol == "/":
                break

            # update melody
            melody.append(output_symbol)

        return melody


    def _sample_with_temperature(self, probabilites, temperature):
        """Samples an index from a probability array reapplying softmax using temperature
        :param predictions (nd.array): Array containing probabilities for each of the possible outputs.
        :param temperature (float): Float in interval [0, 1]. Numbers closer to 0 make the model more deterministic.
            A number closer to 1 makes the generation more unpredictable.
        :return index (int): Selected output symbol
        """
        predictions = np.log(probabilites) / temperature
        probabilites = np.exp(predictions) / np.sum(np.exp(predictions))

        choices = range(len(probabilites)) # [0, 1, 2, 3]
        index = np.random.choice(choices, p=probabilites)

        return index


    def save_melody(self, melody, step_duration=0.25, format="midi", file_name="mel.mid"):
        """Converts a melody into a MIDI file
        :param melody (list of str):
        :param min_duration (float): Duration of each time step in quarter length
        :param file_name (str): Name of midi file
        :return:
        """

        # create a music21 stream
        stream = m21.stream.Stream()

        start_symbol = None
        step_counter = 1

        # parse all the symbols in the melody and create note/rest objects
        for i, symbol in enumerate(melody):

            # handle case in which we have a note/rest
            if symbol != "_" or i + 1 == len(melody):

                # ensure we're dealing with note/rest beyond the first one
                if start_symbol is not None:

                    quarter_length_duration = step_duration * step_counter # 0.25 * 4 = 1

                    # handle rest
                    if start_symbol == "r":
                        m21_event = m21.note.Rest(quarterLength=quarter_length_duration)

                    # handle note
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)

                    stream.append(m21_event)

                    # reset the step counter
                    step_counter = 1

                start_symbol = symbol

            # handle case in which we have a prolongation sign "_"
            else:
                step_counter += 1

        # write the m21 stream to a midi file
        stream.write(format, file_name)
        stream = sitarmusic(stream)

In [10]:
mg = MelodyGenerator()

In [15]:
# Store this code in 'app.py' file

from flask import Flask, render_template, request, redirect, url_for, session
from flask_mysqldb import MySQL
import MySQLdb.cursors
import re
import shutil

app = Flask(__name__)

MODEL_PATH = 'model.h5'
model = load_model(MODEL_PATH)
app.secret_key = 'your secret key'

app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = 'toor'
app.config['MYSQL_DB'] = 'login'

mysql = MySQL(app)

@app.route('/')
@app.route('/login', methods =['GET', 'POST'])
def login():
	msg = ''
	if request.method == 'POST' and 'username' in request.form and 'password' in request.form:
		username = request.form['username']
		password = request.form['password']
		cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
		cursor.execute('SELECT * FROM accounts WHERE username = % s AND password = % s', (username, password, ))
		account = cursor.fetchone()
		if account:
			session['loggedin'] = True
			session['id'] = account['id']
			session['username'] = account['username']
			msg = 'Logged in successfully !'
			return render_template('index.html', msg = msg)
		else:
			msg = 'Incorrect username / password !'
	return render_template('login.html', msg = msg)

@app.route('/logout')
def logout():
	session.pop('loggedin', None)
	session.pop('id', None)
	session.pop('username', None)
	return redirect(url_for('login'))


@app.route('/home')
def home():
	return render_template('index.html')



@app.route('/about')
def about():
	return render_template('about.html')



@app.route('/register', methods =['GET', 'POST'])
def register():
	msg = ''
	if request.method == 'POST' and 'username' in request.form and 'password' in request.form and 'email' in request.form :
		username = request.form['username']
		password = request.form['password']
		email = request.form['email']
		cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
		cursor.execute('SELECT * FROM accounts WHERE username = % s', (username, ))
		account = cursor.fetchone()
		if account:
			msg = 'Account already exists !'
		elif not re.match(r'[^@]+@[^@]+\.[^@]+', email):
			msg = 'Invalid email address !'
		elif not re.match(r'[A-Za-z0-9]+', username):
			msg = 'Username must contain only characters and numbers !'
		elif not username or not password or not email:
			msg = 'Please fill out the form !'
		else:
			cursor.execute('INSERT INTO accounts VALUES (NULL, % s, % s, % s)', (username, password, email, ))
			mysql.connection.commit()
			msg = 'You have successfully registered !'
			return render_template('index.html', msg = msg)
	elif request.method == 'POST':
		msg = 'Please fill out the form !'
	return render_template('register.html', msg = msg)

@app.route('/yaman')
def yaman():
	return render_template('raagas/yaman.html')

@app.route('/bhoopali')
def bhoopali():
	return render_template('raagas/bhoopali.html')

@app.route('/bhairavi')
def bhairavi():
	return render_template('raagas/bhairavi.html')

@app.route('/komal_re_asavari')
def komal_re_asavari():
	return render_template('raagas/komal_re_asavari.html')

@app.route('/generate' , methods =['GET', 'POST'])
def generate():
	if request.method == 'POST' and 'hidden_seed' in request.form:
		seed= request.form['hidden_seed']
		melody = mg.generate_melody(seed, 600, SEQUENCE_LENGTH, 0.5)
		mg.save_melody(melody)
        

# absolute path
#src_path = r"E:\pynative\reports\sales.txt"
#dst_path = r"E:\pynative\account\sales.txt"
#shutil.move(src_path, dst_path)
		shutil.move("mel.mid", "static/audio/mel.mid")
		music_path="static/audio/mel.mid"
		#playmidi('mel.mid')
		#IPython.display.Audio("yaman23.wav")
		return render_template('generate1.html', music_path=music_path)
    
	return render_template('generate.html')




In [16]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/May/2022 14:20:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 14:20:55] "GET /static/index_style.css HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 14:20:55] "GET /static/bg203.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 14:21:01] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 14:21:01] "GET /static/index_style.css HTTP/1.1" 304 -
127.0.0.1 - - [05/May/2022 14:21:02] "GET /static/logo3.PNG HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 14:21:02] "GET /static/bg18.webp HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 14:21:06] "GET /logout HTTP/1.1" 302 -
127.0.0.1 - - [05/May/2022 14:21:06] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 14:21:06] "GET /static/index_style.css HTTP/1.1" 304 -
127.0.0.1 - - [05/May/2022 14:21:06] "GET /static/bg203.jpg HTTP/1.1" 304 -
127.0.0.1 - - [05/May/2022 14:24:59] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 14:24:59] "GET /static/index_style.css H

127.0.0.1 - - [05/May/2022 15:07:10] "GET /static/video/Raag%20Asavari.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [05/May/2022 15:07:28] "GET /yaman HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2022 15:07:28] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [05/May/2022 15:07:28] "GET /static/logo3.PNG HTTP/1.1" 304 -
127.0.0.1 - - [05/May/2022 15:07:28] "GET /static/audio/Raag%20Yaman%20-%20Raag%20Ras%20Barase%20-%20NCERT%20(mp3cut.net).wav HTTP/1.1" 206 -
127.0.0.1 - - [05/May/2022 15:07:28] "GET /static/audio/Raag%20Yaman%20-%20Raag%20Ras%20Barase%20-%20NCERT.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [05/May/2022 15:07:28] "GET /static/video/yaman%20bandish.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [05/May/2022 15:07:28] "GET /static/audio/Raag%20Yaman%20-%20Raag%20Ras%20Barase%20-%20NCERT%20(mp3cut.net).wav HTTP/1.1" 206 -
127.0.0.1 - - [05/May/2022 15:07:28] "GET /static/audio/yaman23.wav HTTP/1.1" 206 -
127.0.0.1 - - [05/May/2022 15:07:29] "GET /static/video/yaman%20bandish.mp4 HTTP/1.1" 206 -
127.0.0.1 - -

In [5]:
mg = MelodyGenerator()
seed = "49 r _ _ r r r r r r _ r _ _ r r _ r 65"
seed2 ="96 _ 77 _ 77 105 r _ _ 76 r r _ _ r _ _ 107 _ 76 _ r 98 r _ r _ _"
seed3="96 _ 77 _ 77 105 r _ _ 76 r r _ _ r _ _ 107 _ 76 _ r 98 _ _ r _ 102 98 _ 74 96 96 "
seed4="88 r r _ r r _ 82 82 r 64 r r _ 83 r r _ 72 _ 85 r r 64 82 r _ r _ r _ r _ _ 71 _ 91 _ 84 60 r _ r _ r _ 60"
seed5="74 _ 82 82 r 64 83 72 _ 64 64 96 r 94 80 80 80 "
seed6="84 94 r _ r r _ r r 79 84 r _ _ r r r r r r 81 r 81 r 81 r 93 r r r 81 r _ r r _ _ 93 r r _ _ r 81 81 74 r"
#saregama 
seed7 = "60 62 64 _ 64 _ _ 67 69 71 72 _ 60 "
seed8="76 _ 83 92 _ r 100 74 _ r _ 60 _ _ 79 _ 84 r _ _ _ 91"
melody = mg.generate_melody(seed7, 600, SEQUENCE_LENGTH, 0.5)
print(melody)
mg.save_melody(melody)

['60', '62', '64', '_', '64', '_', '_', '67', '69', '71', '72', '_', '60', '83', '83', '83', '83', '83', '80', '83', '80', '83', '83', '83', '80', '83', '83', '80', '83', '83', '83', '83', '80', '91', '80', '83', '83', '61', '83', '80', '80', '83', '80', '80', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '83', '83', '83', '80', '83', '83', '83', '47', '83', '69', '83', '80', '83', '83', '80', '83', '83', '80', '83', '80', '83', '69', '83', '80', '80', '83', '80', '80', '83', '80', '80', '83', '83', '83', '69', '83', '80', '83', '71', '83', '83', '83', '83', '83', '83', '80', '71', '83', '83', '83', '80', '83', '80', '80', '83', '80', '71', '83', '80', '

In [28]:
from music21 import midi
from music21 import *
def playmidi(filename):
    mf=midi.MidiFile()
    mf.open(filename)
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    s = converter.parse(s)

    for el in s.recurse():
         if 'Instrument' in el.classes: # or 'Piano'
            el.activeSite.replace(el, instrument.Sitar())

    s.write('midi',filename )
    s.show('midi')
    
    
    

    

In [29]:
playmidi('sitar melody 4.mid')

AttributeError: 'Score' object has no attribute 'startswith'

In [18]:
import shutil

# absolute path
#src_path = r"E:\pynative\reports\sales.txt"
#dst_path = r"E:\pynative\account\sales.txt"
#shutil.move(src_path, dst_path)
shutil.move("mel.mid", "static/audio/mel.mid")

'static/audio/mel.mid'